[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/andreagalle-leonardo/iot-security-lab/blob/main/labs/signature/ecdsa_simulation.ipynb)

# Digital Signatures Simulation (ECDSA)

While hashing ensures data *integrity* (it hasn't changed), it doesn't prove *who* sent the data. 

**Digital Signatures** solve the problem of **Authenticity** and **Non-Repudiation**.

In this notebook, we will simulate an IoT scenario where a device (Alice) signs a critical sensor reading, and a gateway (Bob) verifies that the message truly came from Alice and was not tampered with by an attacker (Mallory).

We will use **ECDSA (Elliptic Curve Digital Signature Algorithm)**, the standard for modern IoT security.

### The Core Concept:
1.  **Signing (Private Key):** Done by the sender. It's like applying a unique wax seal that only you possess.
2.  **Verifying (Public Key):** Done by anyone. It's like checking the wax seal against a known image of your family crest to confirm it's genuine.

In [ ]:
!pip install cryptography

In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.exceptions import InvalidSignature

# Helper function to print long byte strings neatly
def print_hex(label, data):
    print(f"{label:<20}: {data.hex()[:30]}... (truncated)")

## Step 1: Key Generation (Alice)

Just like in encryption, we need a keypair. 

* **The Private Key:** Alice must keep this absolutely SECRET. She uses it to create signatures. If stolen, anyone can impersonate her.
* **The Public Key:** Alice shares this with the world (including Bob). It is used to verify signatures made by her private key.

In [ ]:
# Generate Alice's private key using the SECP256R1 elliptic curve
alice_private_key = ec.generate_private_key(ec.SECP256R1())

# Derive the public key to share with Bob
alice_public_key = alice_private_key.public_key()

print("✅ Alice's keypair generated successfully.")

# Let's visualize the public key (this is what travels over the network)
public_pem = alice_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)
print("\n--- Alice's Public Key (Shared) ---")
print(public_pem.decode())

## Step 2: Signing a Message (Alice)

Alice wants to send a critical command: `"ALERT: HIGH TEMP DETECTED"`.
To prove it's from her, she signs it. 

**How it works internally:**
The signing function first calculates the **Hash (SHA-256)** of the message, and then performs a complex mathematical operation on that hash using the **Private Key**.

In [ ]:
# The critical message (must be bytes)
message = b"ALERT: HIGH TEMP DETECTED"

print(f"Original Message: {message.decode()}\n")

# Alice signs the message using her PRIVATE key
# We specify ECDSA as the algorithm and SHA256 as the internal hashing mechanism
signature = alice_private_key.sign(
    message,
    ec.ECDSA(hashes.SHA256())
)

# The signature is just a block of bytes, usually attached to the message
print_hex("Generated Signature", signature)

print("\n✅ Message signed. Alice sends both the [Message] and the [Signature] to Bob.")

## Step 3: Verifying the Signature (Bob)

Bob receives the message and the signature. He knows Alice's Public Key.

He runs the verification function. This function checks if the signature mathematically corresponds to *both* the message contents and Alice's public key.

If it works, Bob knows two things:
1.  **Authenticity:** Only someone with Alice's private key could have created this.
2.  **Integrity:** The message has not been changed since it was signed.

In [ ]:
print("--- Bob attempts verification ---")

try:
    # Bob uses Alice's PUBLIC key to verify
    alice_public_key.verify(
        signature, 
        message,
        ec.ECDSA(hashes.SHA256())
    )
    print("✅ SUCCESS: Signature is valid. The message is authentic and untampered.")
except InvalidSignature:
    print("❌ FAILURE: Signature is invalid!")

## Step 4: The Attack Scenario - Tampering (Mallory)

What if Mallory intercepts the message in transit and changes it? Let's see if the signature still works.

Mallory changes "HIGH TEMP" to "LOW TEMP" to trick the gateway.

In [ ]:
print("--- Mallory tampers with the message ---")

# The message changes, but Mallory still attaches Alice's original signature
tampered_message = b"ALERT: LOW TEMP DETECTED"
print(f"Tampered Message: {tampered_message.decode()}\n")

try:
    # Bob tries to verify the NEW message with the OLD signature
    alice_public_key.verify(
        signature, 
        tampered_message,
        ec.ECDSA(hashes.SHA256())
    )
    print("✅ SUCCESS: Signature is valid.")
except InvalidSignature:
    print("❌ FAILURE: Verification failed. The message does not match the signature!")
    print("Bob detects the tampering and rejects the command.")

## Step 5: The Attack Scenario - Impersonation (Mallory)

Mallory wants to send a fake command and pretend it's from Alice.
Mallory has her *own* private key, but she doesn't have Alice's.

In [ ]:
print("--- Mallory tries to forge a signature ---")

# Mallory generates her own keys
mallory_private_key = ec.generate_private_key(ec.SECP256R1())

fake_message = b"ALERT: SHUTDOWN SYSTEM"

# Mallory signs it with HER private key
forged_signature = mallory_private_key.sign(
    fake_message,
    ec.ECDSA(hashes.SHA256())
)
print("Mallory sends fake message signed with her own key.\n")

try:
    # Bob thinks it's from Alice, so he uses ALICE'S public key to verify
    alice_public_key.verify(
        forged_signature, 
        fake_message,
        ec.ECDSA(hashes.SHA256())
    )
    print("✅ SUCCESS: Signature is valid.")
except InvalidSignature:
    print("❌ FAILURE: Verification failed. This signature was not created by Alice's private key!")
    print("Bob detects the impersonation attempt.")

## Summary

You have successfully demonstrated that an ECDSA Digital Signature guarantees:

1.  **Integrity:** If the message changes by even one bit, the verification fails (Step 4).
2.  **Authenticity:** If the message is signed by the wrong private key, the verification fails (Step 5).